In [3]:
import copy
import argparse
import pandas as pd
import random

In [490]:
df =pd.read_excel('Documents/matrix.xlsx')

In [491]:
df

,"i,j",0,5,16,24,43,50,52,56,59,68,89,96,100
0,0,0,7,8,14,21,24,11,11,14,17,26,27,31
1,5,7,0,11,11,20,19,4,4,7,10,19,20,24
2,16,8,11,0,8,15,18,13,11,14,9,18,19,23
3,24,14,11,8,0,13,12,13,11,14,9,12,13,17
4,43,21,20,15,13,0,3,22,20,23,18,11,6,10
5,50,24,19,18,12,3,0,21,19,22,17,10,3,7
6,52,11,4,13,13,22,21,0,2,3,10,19,20,22
7,56,11,4,11,11,20,19,2,0,3,8,17,18,22
8,59,14,7,14,14,23,22,3,3,0,9,16,21,19
9,68,17,10,9,9,18,17,10,8,9,0,13,16,16


In [808]:
def generate_neighbours(df):
    dict_of_neighbours = {}
    for idx, row in df.iterrows():
        ls = []
        for j in df.columns[1:]:
            if row['i,j'] != int(j):
                ls.append([int(j), df.at[idx,j]])    
        dict_of_neighbours[row['i,j']] = ls
 
    return dict_of_neighbours

dict_of_neighbours = generate_neighbours(df)

In [809]:
def generate_first_solution(df, dict_of_neighbours):
    first_solution = list(df['i,j'].values[1:])
    random.shuffle(first_solution)
    first_solution.insert(0, 0)
    first_solution.append(0)
    
    distance = 0
    for i in first_solution:
        next_point = first_solution[first_solution.index(16) +1]
        for j in dict_of_neighbours[i]:
            if j[0]== next_point:
                distance = distance + int(j[1])
    
    return first_solution, distance


In [810]:
def neighbor_structure(solution):
    neighbor_table = []
    for i in range(0,len(solution[1:-1])):
        for j in range(i+1, len(solution[1:-1])):
            neighbor_table.append([solution[1:-1][i],solution[1:-1][j], 0])
    return neighbor_table

In [811]:
def swap(i,j, solution):
    neighbor = solution.copy()
    pos_i = solution.index(i)
    pos_j = solution.index(j)
    temp=neighbor[pos_i]
    neighbor[pos_i]=neighbor[pos_j]
    neighbor[pos_j]=temp
    return neighbor

In [812]:
def cal_cost(solution):
    distance = 0
    for i in solution[:-1]:
        next_point = solution[solution.index(i) +1]
        for j in dict_of_neighbours[i]:
            if j[0]== next_point:
                distance = distance + int(j[1])
    return distance

In [813]:
def search_in_table(table, point):
    for idx, row in table.iterrows():
        if set([row['i'],row['j']]) == set(point):
            table.iloc[idx]['tabu_list'] =  1     
    return table

In [814]:
def update_table(table):
    for idx, row in table.iterrows():
        if row['tabu_list'] !=0:
            table.iloc[idx]['tabu_list'] +=1
    return table

In [815]:
def tabu_search(neighbor_table,
                dict_of_neighbours,
                iterations,
                number_of_neighbors,
                size,
                first_solution,
                distance_of_first_solution ):
    
    solution = first_solution
    best_solution = first_solution
    best_cost = distance_of_first_solution
    count = 0
    tabu_list = {}
    
    structure = pd.DataFrame (neighbor_table, columns = ['i', 'j','tabu_list'])
    #print(structure)
    while count <= iterations:
       
        # randomly take the number of neighbors
        temp = structure[structure['tabu_list']==0].sample(n= number_of_neighbors)
        
        solution_space = []
        points = []
        for idx, row in temp.iterrows():
            a = swap(row['i'], row['j'], solution)
            solution_space.append(a)
            points.append([row['i'], row['j']])

        # check the cost of all swaps and take only the min.
        cost = [cal_cost(sol) for sol in solution_space]
        # get the best swap and solution
        point = points[cost.index(min(cost))]
        solution = solution_space[cost.index(min(cost))]
        if int(min(cost)) < best_cost:
            best_cost = int(min(cost))
            best_solution = solution
        #print(solution, point, best_cost)
         #update table
        structure = update_table(structure)
        structure = search_in_table(structure, point)
        if len(structure['tabu_list']) >= size:
            structure['tabu_list'] = structure['tabu_list'].replace(size, 0)
        
        count +=1
        #print(structure[structure['tabu_list']!= 0])
    return best_solution, best_cost     
        

In [830]:

for i in range(10):
    iterations = 5
    size =1
    number_of_neighbors = 3
    first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
    neighbor_table= neighbor_structure(first_solution)
    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)

    print(i, f"Best solution: {best_sol}, with total distance: {best_cos}.")
    i +=1


0 Best solution: [0, 59, 68, 24, 50, 43, 16, 52, 5, 56, 100, 89, 96, 0], with total distance: 148.
1 Best solution: [0, 56, 52, 68, 16, 89, 100, 59, 5, 24, 50, 43, 96, 0], with total distance: 142.
2 Best solution: [0, 43, 68, 52, 89, 100, 96, 50, 24, 56, 16, 5, 59, 0], with total distance: 148.
3 Best solution: [0, 52, 50, 43, 16, 24, 100, 96, 56, 5, 68, 89, 59, 0], with total distance: 154.
4 Best solution: [0, 16, 5, 52, 68, 43, 59, 100, 96, 50, 24, 89, 56, 0], with total distance: 152.
5 Best solution: [0, 56, 52, 5, 43, 96, 24, 50, 89, 100, 68, 59, 16, 0], with total distance: 132.
6 Best solution: [0, 5, 68, 24, 100, 50, 43, 96, 89, 52, 59, 56, 16, 0], with total distance: 112.
7 Best solution: [0, 5, 68, 89, 100, 96, 24, 50, 43, 56, 52, 59, 16, 0], with total distance: 116.
8 Best solution: [0, 50, 100, 89, 96, 16, 68, 24, 52, 59, 43, 5, 56, 0], with total distance: 158.
9 Best solution: [0, 16, 43, 56, 96, 50, 24, 89, 100, 59, 5, 68, 52, 0], with total distance: 152.


In [832]:
iterations = 10
size =3
number_of_neighbors = 6
first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
neighbor_table= neighbor_structure(first_solution)
for i in range(10):
    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)
    
    print(f"Best solution: {best_sol}, with total distance: {best_cos}.")
    i = i +1

Best solution: [0, 16, 52, 59, 56, 89, 100, 96, 43, 50, 24, 68, 5, 0], with total distance: 102.
Best solution: [0, 16, 96, 100, 68, 52, 59, 89, 43, 50, 24, 5, 56, 0], with total distance: 128.
Best solution: [0, 5, 59, 68, 89, 24, 16, 96, 100, 50, 43, 56, 52, 0], with total distance: 122.
Best solution: [0, 5, 59, 56, 52, 16, 43, 100, 96, 50, 89, 24, 68, 0], with total distance: 112.
Best solution: [0, 5, 52, 59, 56, 68, 16, 24, 96, 89, 100, 50, 43, 0], with total distance: 102.
Best solution: [0, 56, 5, 59, 52, 24, 68, 89, 100, 96, 50, 43, 16, 0], with total distance: 100.
Best solution: [0, 16, 24, 68, 56, 52, 59, 89, 100, 96, 43, 50, 5, 0], with total distance: 100.
Best solution: [0, 5, 56, 89, 100, 43, 96, 50, 52, 68, 59, 24, 16, 0], with total distance: 124.
Best solution: [0, 5, 56, 52, 59, 89, 96, 100, 50, 43, 24, 68, 16, 0], with total distance: 94.
Best solution: [0, 56, 52, 59, 5, 68, 100, 96, 89, 24, 50, 43, 16, 0], with total distance: 112.


In [834]:
iterations = 20
size =6
number_of_neighbors = 12
first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
neighbor_table= neighbor_structure(first_solution)
for i in range(10):
    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)
    
    print(f"Best solution: {best_sol}, with total distance: {best_cos}.")

Best solution: [0, 16, 68, 89, 96, 100, 43, 50, 24, 5, 52, 59, 56, 0], with total distance: 100.
Best solution: [0, 16, 24, 89, 100, 96, 43, 50, 56, 68, 59, 52, 5, 0], with total distance: 98.
Best solution: [0, 5, 52, 56, 59, 43, 50, 96, 100, 89, 24, 68, 16, 0], with total distance: 94.
Best solution: [0, 16, 68, 89, 100, 43, 50, 96, 24, 59, 56, 52, 5, 0], with total distance: 96.
Best solution: [0, 16, 43, 50, 96, 100, 89, 68, 56, 59, 52, 5, 24, 0], with total distance: 96.
Best solution: [0, 5, 56, 59, 52, 68, 100, 96, 50, 43, 89, 24, 16, 0], with total distance: 92.
Best solution: [0, 16, 68, 24, 89, 43, 50, 96, 100, 59, 52, 56, 5, 0], with total distance: 94.
Best solution: [0, 5, 56, 52, 59, 89, 100, 96, 50, 43, 24, 68, 16, 0], with total distance: 88.
Best solution: [0, 16, 68, 24, 50, 43, 100, 96, 89, 59, 52, 56, 5, 0], with total distance: 96.
Best solution: [0, 5, 68, 50, 43, 96, 100, 89, 59, 52, 56, 24, 16, 0], with total distance: 102.


In [836]:
iterations = 25
size =8
number_of_neighbors = 16
first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
neighbor_table= neighbor_structure(first_solution)
for i in range(10):
    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)
    
    print(f"Best solution: {best_sol}, with total distance: {best_cos}.")

Best solution: [0, 43, 50, 96, 100, 89, 16, 68, 24, 59, 56, 52, 5, 0], with total distance: 104.
Best solution: [0, 16, 68, 89, 100, 96, 50, 43, 24, 56, 59, 52, 5, 0], with total distance: 88.
Best solution: [0, 16, 24, 50, 43, 96, 100, 89, 68, 59, 52, 56, 5, 0], with total distance: 86.
Best solution: [0, 16, 24, 50, 43, 96, 100, 89, 68, 52, 59, 56, 5, 0], with total distance: 88.
Best solution: [0, 16, 24, 43, 50, 96, 100, 89, 68, 56, 52, 59, 5, 0], with total distance: 86.
Best solution: [0, 16, 43, 50, 96, 100, 89, 24, 68, 52, 56, 59, 5, 0], with total distance: 90.
Best solution: [0, 16, 24, 89, 100, 96, 50, 43, 68, 59, 56, 52, 5, 0], with total distance: 88.
Best solution: [0, 24, 50, 96, 100, 89, 43, 16, 68, 59, 56, 52, 5, 0], with total distance: 100.
Best solution: [0, 16, 24, 50, 43, 96, 100, 89, 68, 56, 59, 5, 52, 0], with total distance: 94.
Best solution: [0, 24, 89, 100, 96, 50, 43, 5, 56, 52, 59, 68, 16, 0], with total distance: 98.


In [851]:
for i in range(10):
    iterations = 25
    size =10
    number_of_neighbors = 30
    first_solution, distance_of_first_solution = generate_first_solution(df, dict_of_neighbours)
    neighbor_table= neighbor_structure(first_solution)

    best_sol, best_cos = tabu_search(neighbor_table,
                    dict_of_neighbours,
                    iterations,
                    number_of_neighbors,
                    size,
                    first_solution,
                    distance_of_first_solution)
    
    print(f"Best solution: {best_sol}, with total distance: {best_cos}.")

Best solution: [0, 16, 5, 52, 59, 56, 68, 89, 100, 96, 50, 43, 24, 0], with total distance: 94.
Best solution: [0, 5, 52, 56, 59, 68, 89, 100, 96, 43, 50, 24, 16, 0], with total distance: 86.
Best solution: [0, 16, 68, 59, 56, 52, 5, 24, 89, 100, 96, 50, 43, 0], with total distance: 96.
Best solution: [0, 5, 52, 59, 56, 68, 89, 50, 100, 96, 43, 24, 16, 0], with total distance: 94.
Best solution: [0, 5, 52, 56, 59, 24, 68, 89, 100, 96, 50, 43, 16, 0], with total distance: 92.
Best solution: [0, 5, 52, 59, 56, 68, 89, 100, 50, 43, 96, 24, 16, 0], with total distance: 90.
Best solution: [0, 16, 24, 68, 59, 56, 52, 5, 89, 100, 96, 50, 43, 0], with total distance: 100.
Best solution: [0, 5, 59, 52, 56, 68, 43, 50, 96, 100, 89, 24, 16, 0], with total distance: 90.
Best solution: [0, 16, 24, 43, 50, 96, 100, 89, 68, 59, 56, 52, 5, 0], with total distance: 84.
Best solution: [0, 5, 56, 52, 59, 68, 89, 100, 96, 50, 43, 24, 16, 0], with total distance: 84.
